# Geospatial Analysis — Hospitals in Peru (GeoPandas)

**Author:** _Your Name_

**Objective:** Clean IPRESS data and produce GeoPandas outputs for team visualizations (Folium & Streamlit).

## What this notebook does
1. **Data cleaning** of IPRESS (MINSA) operational hospitals:

   - Keep only operational hospitals (filter by *situacion/condicion* containing "FUNCION").

   - Rename `NORTE -> LATITUD` and `ESTE -> LONGITUD`.

   - Convert coordinates to numeric and keep valid Peru ranges.

   - Save → `data/processed/hospitales_clean.csv`.

2. **Task 1 — Static Maps (Hospital count by district):**

   - Spatial join Hospitals ↔ District polygons.

   - Export GeoJSONs: `district_counts.geojson`, `district_zero.geojson`, `district_top10.geojson`.

3. **Task 2 — Department-level analysis:**

   - Aggregate counts by department.

   - Save → `department_counts.csv`.

4. **Task 3 — Proximity (10 km) using Population Centers:**

   - For **Lima** and **Loreto**, compute 10 km buffers around each population center and count hospitals inside.

   - Export buffers (min & max density) and summary CSVs for Folium/Streamlit.

   
> All final files are saved in the `salida/` folder for Folium (choropleths, clusters, and 10 km circles) and for Streamlit (tabs 2 & 3).


## 0) Setup & Imports

In [ ]:
# Standard imports
import os
import re
import unicodedata
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Make folders if missing
os.makedirs("data/processed", exist_ok=True)
os.makedirs("salida", exist_ok=True)

# Paths (relative to repo root)
P_IPRESS = "data/ipress_raw.csv"          # <-- Your raw IPRESS CSV (without tildes ideally)
P_DIST   = "data/distritos.zip"           # <-- Zipped shapefile: distritos (INEI)
P_CP     = "data/centros_poblados.zip"    # <-- Zipped shapefile: centros poblados (INEI)


## 1) Helper functions

In [ ]:
# --- Helper: normalize column names (lowercase, ascii, underscores) ---
def normalize_cols(df):
    df = df.copy()
    df.columns = (df.columns
                  .str.strip()
                  .map(lambda x: unicodedata.normalize("NFKD", x))
                  .str.encode("ascii", "ignore").str.decode("utf-8")
                  .str.lower()
                  .str.replace(r"[^a-z0-9]+", "_", regex=True)
                  .str.strip("_"))
    return df

# --- Helper: try to find a column by candidates / substrings (case-insensitive) ---
def find_col(columns, candidates):
    cols = [c.lower() for c in columns]
    for cand in candidates:
        lc = cand.lower()
        for c in columns:
            if lc == c.lower():
                return c
        # substring fallback
        for c in columns:
            if lc in c.lower():
                return c
    return None

# --- Helper: strip accents and uppercase for robust comparisons ---
def up_noacc(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.upper().strip()



## 2) Load & clean IPRESS (MINSA)

In [ ]:
# 2.1) Load raw IPRESS data (CSV). If you have Excel, change to pd.read_excel.
df_raw = pd.read_csv(P_IPRESS, dtype=str, encoding="utf-8")
print("Raw shape:", df_raw.shape)

# 2.2) Optional: replace comma decimal separators before numeric conversion
for c in ["Latitud","Longitud","NORTE","ESTE"]:
    if c in df_raw.columns and df_raw[c].dtype == "object":
        df_raw[c] = df_raw[c].str.replace(",", ".", regex=False)

# 2.3) Rename columns to standard (handles variants and your 'NORTE'/'ESTE' case)
rename_map = {
    # Spanish originals (no tildes to help matching)
    "Institucion"               : "INSTITUCION",
    "Nombre del Establecimiento": "NOMBRE_DEL_ESTABLECIMIENTO",
    "Nombre d"                  : "NOMBRE_DEL_ESTABLECIMIENTO",
    "Clasificacion"             : "CLASIFICACION",
    "Clasifica"                 : "CLASIFICACION",
    "Departamento"              : "DEPARTAMENTO",
    "Departam"                  : "DEPARTAMENTO",
    "Provincia"                 : "PROVINCIA",
    "Distrito"                  : "DISTRITO",
    "Estado"                    : "ESTADO",
    "Latitud"                   : "LATITUD",
    "Longitud"                  : "LONGITUD",
    "NORTE"                     : "LATITUD",
    "ESTE"                      : "LONGITUD",
}
# Apply renaming only for present columns
df = df_raw.rename(columns={k:v for k,v in rename_map.items() if k in df_raw.columns}).copy()

# 2.4) Filter operational hospitals (search in any column named 'situacion' or 'condicion')
op_cols = [c for c in df.columns if re.search(r"(situaci|condici)", c, flags=re.I)]
if op_cols:
    mask = False
    for c in op_cols:
        mask = mask | df[c].astype(str).str.contains("FUNCION", case=False, na=False)
    df = df[mask].copy()

# 2.5) Keep only the required minimal columns
keep = ["INSTITUCION","NOMBRE_DEL_ESTABLECIMIENTO","CLASIFICACION",
        "DEPARTAMENTO","PROVINCIA","DISTRITO","ESTADO","LATITUD","LONGITUD"]
present = [c for c in keep if c in df.columns]
missing  = [c for c in keep if c not in df.columns]
print("Present columns:", present)
if missing:
    print("WARNING - Missing expected columns:", missing)

df = df[present].copy()

# 2.6) Convert coordinates to numeric and keep valid Peru ranges
for c in ["LATITUD","LONGITUD"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Peru rough bounding box: lat (−20..5), lon (−90..−60)
if set(["LATITUD","LONGITUD"]).issubset(df.columns):
    df = df[
        df["LATITUD"].between(-20, 5) &
        df["LONGITUD"].between(-90, -60)
    ].dropna(subset=["LATITUD","LONGITUD"])

print("Clean shape:", df.shape)
df.head(3)


### 2.7) Save cleaned hospitals → `data/processed/hospitales_clean.csv`

In [ ]:
out_clean = "data/processed/hospitales_clean.csv"
df.to_csv(out_clean, index=False, encoding="utf-8")
print("Saved:", out_clean, "rows:", len(df))

## 3) Hospitals → GeoDataFrame (EPSG:4326)

In [ ]:
# Ensure required columns for geometry
assert "LATITUD" in df.columns and "LONGITUD" in df.columns, "LATITUD/LONGITUD missing after cleaning."

gdf_h = gpd.GeoDataFrame(
    df.copy(),
    geometry=gpd.points_from_xy(df["LONGITUD"], df["LATITUD"]),
    crs="EPSG:4326"
)
gdf_h.head(2)

## 4) Read Districts & Population Centers shapefiles (INEI)

In [ ]:
# Read zipped shapefiles directly
gdf_dist = gpd.read_file(f"zip://{P_DIST}")
if gdf_dist.crs is None or gdf_dist.crs.to_string() != "EPSG:4326":
    gdf_dist = gdf_dist.to_crs("EPSG:4326")

gdf_cp = gpd.read_file(f"zip://{P_CP}")
if gdf_cp.crs is None or gdf_cp.crs.to_string() != "EPSG:4326":
    gdf_cp = gdf_cp.to_crs("EPSG:4326")

print("Districts:", gdf_dist.shape, gdf_dist.crs)
print("Pop. Centers:", gdf_cp.shape, gdf_cp.crs)

# Try to detect district & department columns in the districts layer
dist_col = find_col(gdf_dist.columns, ["DISTRITO","NOMB_DIST","NOMBDIST","DIST"])
dept_col = find_col(gdf_dist.columns, ["DEPARTAMEN","DEPARTAMENTO","NOMBDEP","DPTO","DEPART"])
print("Detected fields → District:", dist_col, "| Department:", dept_col)
assert dist_col and dept_col, "Cannot detect district/department columns in districts shapefile."


## 5) Task 1 — Hospital Count by District

In [ ]:
# 5.1) Spatial join: assign each hospital to a district polygon
joined = gpd.sjoin(
    gdf_h,
    gdf_dist[[dist_col, dept_col, "geometry"]],
    how="left",
    predicate="within"
)

# 5.2) Count hospitals by district (with department as grouping)
dist_counts = (joined
               .groupby([dept_col, dist_col], dropna=False)
               .size().reset_index(name="hospitales"))

# 5.3) Merge counts back to district polygons
gdf_dist_cnt = gdf_dist.merge(dist_counts, on=[dept_col, dist_col], how="left")
gdf_dist_cnt["hospitales"] = gdf_dist_cnt["hospitales"].fillna(0).astype(int)

# 5.4) Export GeoJSONs for Folium
gdf_dist_cnt.to_file("salida/district_counts.geojson", driver="GeoJSON")
gdf_dist_cnt[gdf_dist_cnt["hospitales"]==0].to_file("salida/district_zero.geojson", driver="GeoJSON")
gdf_dist_cnt.sort_values("hospitales", ascending=False).head(10)            .to_file("salida/district_top10.geojson", driver="GeoJSON")

print("Saved: salida/district_counts.geojson, district_zero.geojson, district_top10.geojson")
gdf_dist_cnt[["hospitales"]].describe()


## 6) Task 2 — Department-level Analysis

In [ ]:
# 6.1) Aggregate by department using hospitals table (textual field from IPRESS)
dep_text_col = find_col(gdf_h.columns, ["DEPARTAMENTO","DEPARTAMEN","DEPTO","REGION","AMBITO"])
assert dep_text_col, "Cannot detect a department-like column in hospitals data."

dep_tbl = (gdf_h.groupby(dep_text_col).size()
           .reset_index(name="hospitales")
           .sort_values("hospitales", ascending=False))

# 6.2) Save CSV for Streamlit (table + chart)
dep_tbl.to_csv("salida/department_counts.csv", index=False, encoding="utf-8")
print("Saved: salida/department_counts.csv")
dep_tbl.head(10)


## 7) Task 3 — Proximity (10 km) using Population Centers

**Regions to analyze:** Lima and Loreto.

**Method:**

- Detect department column in population centers.

- For each region (Lima/Loreto):

  1) Filter population centers in that department.

  2) Reproject to `EPSG:32718` (meters), compute 10 km buffers.

  3) Count hospitals within each buffer.

  4) Export GeoJSONs for the **min** and **max** hospital densities and a summary CSV.


In [ ]:
# 7.1) Detect department column in population centers
cp_dept_col = find_col(gdf_cp.columns, ["DEPARTAMEN","DEPARTAMENTO","NOMBDEP","DPTO","DEPART"])
if not cp_dept_col:
    raise ValueError("Department column not found in Population Centers shapefile. Please inspect gdf_cp.columns.")

# 7.2) Prepare metric projections
h_m  = gdf_h.to_crs("EPSG:32718")
cp_m = gdf_cp.to_crs("EPSG:32718")

regions = ["LIMA", "LORETO"]
summary_rows = []

def export_proximity_for_region(reg_upper):
    # Filter CP by department (accent/case-insensitive)
    sel = gdf_cp[cp_dept_col].map(up_noacc) == reg_upper
    cp_reg4326 = gdf_cp[sel].copy()
    if cp_reg4326.empty:
        print(f"[WARN] No population centers found for department: {reg_upper}")
        return None

    # Work in meters for buffers
    cp_reg = cp_reg4326.to_crs("EPSG:32718").copy()
    # If CP geometry are not points, use centroids
    cp_reg["centroid"] = cp_reg.geometry.centroid
    cp_reg = gpd.GeoDataFrame(cp_reg.drop(columns="geometry"), geometry=cp_reg["centroid"], crs="EPSG:32718")

    # 10 km buffer
    cp_reg["buffer10"] = cp_reg.geometry.buffer(10000)
    buffers = gpd.GeoDataFrame(cp_reg.drop(columns="geometry"), geometry=cp_reg["buffer10"], crs="EPSG:32718")

    # Count hospitals within each buffer
    counts = gpd.sjoin(buffers, h_m, how="left", predicate="contains")\
                .groupby(buffers.index).size().reindex(buffers.index, fill_value=0)
    buffers["hosp_10km"] = counts.values

    # Find extremes
    fewest = buffers.sort_values("hosp_10km", ascending=True).head(1)
    most   = buffers.sort_values("hosp_10km", ascending=False).head(1)

    # Export GeoJSONs (back to EPSG:4326)
    fewest4326 = fewest.to_crs("EPSG:4326")
    most4326   = most.to_crs("EPSG:4326")

    fewest_path = f"salida/proximity_buffers_{reg_upper.lower()}_min.geojson"
    most_path   = f"salida/proximity_buffers_{reg_upper.lower()}_max.geojson"
    fewest4326.to_file(fewest_path, driver="GeoJSON")
    most4326.to_file(most_path, driver="GeoJSON")

    # Also export hospitals inside each selected buffer (for Folium)
    h_in_fewest = gpd.sjoin(gdf_h, fewest4326[["geometry"]], how="inner", predicate="within")
    h_in_most   = gpd.sjoin(gdf_h, most4326[["geometry"]],   how="inner", predicate="within")
    h_in_fewest.to_file(f"salida/proximity_hospitals_{reg_upper.lower()}_min.geojson", driver="GeoJSON")
    h_in_most.to_file(f"salida/proximity_hospitals_{reg_upper.lower()}_max.geojson", driver="GeoJSON")

    print("Saved:", fewest_path, "and", most_path)
    return int(fewest["hosp_10km"].iloc[0]), int(most["hosp_10km"].iloc[0])

for reg in regions:
    res = export_proximity_for_region(reg)
    if res:
        mn, mx = res
        summary_rows.append({"region": reg.title(), "min_hosp_10km": mn, "max_hosp_10km": mx})

summary = pd.DataFrame(summary_rows)
summary.to_csv("salida/proximity_summary.csv", index=False, encoding="utf-8")
summary


## 8) Files produced (for your team)

**Processed base data**

- `data/processed/hospitales_clean.csv`

**Task 1 (district level)**

- `salida/district_counts.geojson`

- `salida/district_zero.geojson`

- `salida/district_top10.geojson`

**Task 2 (department level)**

- `salida/department_counts.csv`

**Task 3 (proximity, 10 km)**

- `salida/proximity_buffers_lima_min.geojson`, `salida/proximity_buffers_lima_max.geojson`

- `salida/proximity_buffers_loreto_min.geojson`, `salida/proximity_buffers_loreto_max.geojson`

- `salida/proximity_hospitals_lima_min.geojson`, `salida/proximity_hospitals_lima_max.geojson`

- `salida/proximity_hospitals_loreto_min.geojson`, `salida/proximity_hospitals_loreto_max.geojson`

- `salida/proximity_summary.csv`

> These are ready to be used by your teammates in **Folium** (choropleths, clusters, 10 km buffers) and **Streamlit** (tabs 2 & 3).


## 9) Quick Visual Previews (optional)

These cells generate **inline previews** so you can see results directly in Jupyter.
They are not required for the export files, but they're helpful for QA.

- Static choropleth preview (district counts) — `matplotlib`
- Department bar chart (Top 10) — `matplotlib`
- Folium preview maps — hospitals sample and Lima proximity buffers


In [ ]:
import matplotlib.pyplot as plt

# 9.1) Static choropleth preview for district counts
fig = plt.figure(figsize=(7, 8))
ax = plt.gca()
_ = gdf_dist_cnt.plot(column="hospitales", legend=True, ax=ax)
ax.set_title("Hospitals per District — preview")
ax.set_axis_off()
plt.show()

# 9.2) Department bar chart (Top 10)
fig = plt.figure(figsize=(7, 6))
ax = plt.gca()
(dep_tbl.head(10).sort_values("hospitales", ascending=True)
 .plot(kind="barh", x=dep_text_col, y="hospitales", ax=ax))
ax.set_title("Hospitals by Department — Top 10 (preview)")
ax.set_xlabel("Hospitals")
ax.set_ylabel("Department")
plt.tight_layout()
plt.show()

In [ ]:
import folium
from folium.plugins import MarkerCluster

# 9.3) Folium — hospital points (sample up to 1000 for speed)
if not gdf_h.empty:
    center = [float(gdf_h['LATITUD'].mean()), float(gdf_h['LONGITUD'].mean())]
    m_points = folium.Map(location=center, zoom_start=5, tiles="cartodbpositron")
    mc = MarkerCluster().add_to(m_points)

    sample_n = min(1000, len(gdf_h))
    for _, r in gdf_h.sample(sample_n, random_state=42).iterrows():
        folium.CircleMarker(
            location=[float(r['LATITUD']), float(r['LONGITUD'])],
            radius=3, fill=True, weight=1,
            popup=str(r.get('NOMBRE_DEL_ESTABLECIMIENTO', 'Hospital'))
        ).add_to(mc)
    m_points
else:
    print("No hospital points to preview.")

In [ ]:
# 9.4) Folium — Lima proximity buffers preview (min & max)
import os
import geopandas as gpd

lima_min_fp = "salida/proximity_buffers_lima_min.geojson"
lima_max_fp = "salida/proximity_buffers_lima_max.geojson"
h_lima_min_fp = "salida/proximity_hospitals_lima_min.geojson"
h_lima_max_fp = "salida/proximity_hospitals_lima_max.geojson"

if all(os.path.exists(p) for p in [lima_min_fp, lima_max_fp, h_lima_min_fp, h_lima_max_fp]):
    lima_min = gpd.read_file(lima_min_fp)
    lima_max = gpd.read_file(lima_max_fp)
    h_lima_min = gpd.read_file(h_lima_min_fp)
    h_lima_max = gpd.read_file(h_lima_max_fp)

    # Map center: mean of all hospital points in these layers (fallback to Peru center if empty)
    if not h_lima_max.empty:
        center = [float(h_lima_max.geometry.y.mean()), float(h_lima_max.geometry.x.mean())]
    else:
        center = [-9.2, -75.0]

    m_lima = folium.Map(location=center, zoom_start=7, tiles="cartodbpositron")
    folium.GeoJson(lima_min).add_to(m_lima)
    folium.GeoJson(lima_max).add_to(m_lima)

    for gdf_pts in [h_lima_min, h_lima_max]:
        for _, r in gdf_pts.iterrows():
            folium.CircleMarker(location=[float(r.geometry.y), float(r.geometry.x)],
                                radius=2, fill=True, weight=0.5).add_to(m_lima)
    m_lima
else:
    print("Lima proximity files not found yet. Run Task 3 cells first.")

## 10) Task 1 — Static Maps (PNG exports)

This section produces the **three static maps** required by Task 1 using **GeoPandas + Matplotlib**, and saves them into the `salida/` folder:

- `map1_district_counts.png` → total public hospitals per district (choropleth).
- `map2_district_zero.png`   → districts with **zero** hospitals highlighted.
- `map3_district_top10.png`  → **Top 10** districts with the highest number of hospitals.

> Run the earlier cells first (especially Task 1 join) so `gdf_dist_cnt` exists.


In [ ]:
import os
import matplotlib.pyplot as plt

os.makedirs("salida", exist_ok=True)

# Safety checks
assert 'gdf_dist_cnt' in globals(), "Run Task 1 cells first to create gdf_dist_cnt."
assert 'dist_col' in globals() and 'dept_col' in globals(), "District/Department fields not detected."

# -------- Map 1: Total hospitals per district (choropleth) --------
fig = plt.figure(figsize=(8, 9))
ax = plt.gca()
gdf_dist_cnt.plot(column="hospitales", legend=True, ax=ax)  # default colormap, no manual colors
ax.set_title("Map 1 — Hospitals per District (Total)")
ax.set_axis_off()
plt.tight_layout()
plt.savefig("salida/map1_district_counts.png", dpi=220)
plt.show()

# -------- Map 2: Highlight districts with zero hospitals --------
gdf_zero = gdf_dist_cnt[gdf_dist_cnt["hospitales"] == 0]

fig = plt.figure(figsize=(8, 9))
ax = plt.gca()
# base layer faint
gdf_dist_cnt.plot(ax=ax, alpha=0.12)            # light background
# overlay zeros (default style, thicker borders to stand out)
gdf_zero.plot(ax=ax, linewidth=1.0)             # defaults; no explicit colors
ax.set_title("Map 2 — Districts with ZERO Hospitals (highlighted)")
ax.set_axis_off()
plt.tight_layout()
plt.savefig("salida/map2_district_zero.png", dpi=220)
plt.show()

# -------- Map 3: Top 10 districts with highest number of hospitals --------
top10 = gdf_dist_cnt.sort_values("hospitales", ascending=False).head(10)

fig = plt.figure(figsize=(8, 9))
ax = plt.gca()
# base layer faint
gdf_dist_cnt.plot(ax=ax, alpha=0.12)
# overlay top10 (default style, thicker borders)
top10.plot(ax=ax, linewidth=1.2)
ax.set_title("Map 3 — Top 10 Districts by Number of Hospitals")
ax.set_axis_off()
plt.tight_layout()
plt.savefig("salida/map3_district_top10.png", dpi=220)
plt.show()

print("Saved PNGs to 'salida/':",
      "\n - map1_district_counts.png",
      "\n - map2_district_zero.png",
      "\n - map3_district_top10.png")